# Исследование уровня потребительской лояльности

**Цель исследования** - определить текущий уровень потребительской лояльности, или NPS, среди клиентов из России.

**Вопросы, интересующие заказчика:**
- Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников?

**Ссылка на проект:** https://public.tableau.com/views/_17045382255270/sheet13?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

## Шаг 1. Подключение к базе  и выгрузка данных

In [9]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [10]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [13]:
query = """
SELECT u.user_id,
       u.lt_day,
       CASE
           WHEN u.lt_day <= 365 THEN 'новый'
           ELSE 'старый'
       END as Is_new,
       u.age,
       CASE
           WHEN u.gender_segment = 0 THEN "мужской"
           WHEN u.gender_segment = 1 THEN "женский"
       END as gender,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       age_s.title as age_segment,
       ts.title as traffic_segment,
       ls.title as lifetime_segment,
       u.nps_score,
       CASE
           WHEN u.nps_score <= 6 THEN 'критики'
           WHEN u.nps_score >= 7 AND u.nps_score <= 8 THEN 'нейтралы'
           WHEN u.nps_score >= 9 AND u.nps_score <= 10 THEN 'cторонники'
       END as nps_group
FROM user AS u
JOIN location AS l ON u.location_id = l.location_id
JOIN age_segment AS age_s ON u.age_gr_id = age_s.age_gr_id
JOIN traffic_segment AS ts ON u.tr_gr_id = ts.tr_gr_id
JOIN lifetime_segment AS ls ON u.lt_gr_id = ls.lt_gr_id
"""

In [14]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,Is_new,age,gender,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,старый,45.0,женский,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,cторонники
1,A001WF,2344,старый,53.0,мужской,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,cторонники
2,A003Q7,467,старый,57.0,мужской,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,cторонники


In [15]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

**Вывод по разделу:** 

В данном шаге сделали подключение к базе данных. Далее с помощью SQL-запроса выгррузили необходимые поля из базы данных. Сохранили CSV-файл и загрузили его в Tableau для дальнейшего проведения исследования.

## Шаг 2. Создание дашборда "Характеристики опрошенных пользователей" в Tableau

В этом шаге мы построили 5 визуализаций и объединили их в дашборд "Характеристики опрошенных пользователей".

В первой визуализации отобразили соотношение полов опрошенных потребителей. Во второй визуализации посмотрели соотношение опрошенных по возрастным группам, в третьей - по новизне потребителя, в четвертой - по городам, и в пятой - по типу операционной системы.

**Выводы по разделу:**

- Большая часть опрошенных - женщины.
- Самая опрашиваемая возрастная группа от 35 до 44 лет.
- Самая популярная ОС среди опрошенных - Android.
- Больше всего опрошенных проживают в Москве, Санкт-Петербурге и Новосибирске.
- Новых пользователей среди опрошенных почти в 5 раз меньше, чем старых.

## Шаг 3. Создание дашборда "Анализ групп потребительской лояльности" в Tableau

На данном этапе были созданы 6 визуализаций и объединены в дашборд "Анализ групп потребительской лояльности".
 
В первой визуализации отобразили процентное соотношение групп лояльности опрошенных. Во второй - лояльность опрошенных по возрастным группам. В третьей - лояльность по полу. В четвертой - лояльность по новизне клиентов. В пятой - лояльность по типу операционной системы. В шестой - лояльность по времени пользования (lifetime).

**Выводы по разделу:**

- Среди опрошенных 52% являются сторонниками, 30% - критиками, 16% - нейтралами.
- Во всех возрастных категориях в процентном соотношении преобладают сторонники, за исключением опрошенных до 16 лет. Больше всего сторонников в группе 66+. Также больше сторонников наблюдается среди женщин и новых потребителей.
- По типу операционной системы в процентном соотношении сторонников больше у неизвестной ОС, но в количественном соотношении по количеству сторонников лидируют Android.
- По времени пользования в процентном соотношении лидирует группа, пользовавшаяся услугами 1 месяц, но в количественном соотношении - группа, пользующаяся услугами более 36 месяцев, что логично.

## Шаг 4. Создание дашборда "Уровень потребительской лояльности (NPS)" в Tableau

На дашборде "Уровень потребительской лояльности (NPS)" отобразили 9 графиков: 

1. Общий NPS
2. NPS по полу
3. NPS по возрасту
4. NPS по новизне
5. NPS по типу операционной системы
6. NPS по городам
7. NPS по времени пользования (lifetime)
8. NPS по типу устройства
9. NPS по трафику

**Выводы по разделу:**

- Общий NPS составляет 21,97%.
- Наибольшие значения NPS в Иваново, Череповце и Саранске.
- Уровень потребительской лояльности наиболее высок среди новых клиентов женского пола  в возрасте 66+ с трафиком 0-0,01, пользующиеся устройствами USB MODEM с операционной системой Symbian OS и с временем пользования в 1 месяц.
- Самый низкий уровень лояльности среди старых клиентов мужского пола в возрасте до 16 лет с трафиком 100+, пользующиеся устройствами WLAN ROUTER и MODEM, операционной системой IOS и временем пользования 36+ месяцев.

## Шаг 5. Создание презентации

В этом этапе объединили 2 наших дашборда в одну презентацию.

**Ссылка на проект:** https://public.tableau.com/views/_17045382255270/sheet13?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link